In [1]:
!pip install django
!pip install nest_asyncio


In [8]:
import os
import sys
import django

# Adjust the Python path if necessary
project_path = '/home/nobel/Desktop/web/Snapfitchat'
sys.path.append(project_path)

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'core.settings')

# Initialize Django
django.setup()


In [ ]:
# Import the models after setting up Django
from django.contrib.auth.models import User
from gym.models import Exercise, UserAssessment, WorkoutSchedule, WorkoutScheduleDay
from openai import OpenAI
from asgiref.sync import sync_to_async
import nest_asyncio
import asyncio
import threading
import json 
import re


# Set your OpenAI API key
client = OpenAI(api_key = 'Your chat gpt key')

elx = []
eld = []

def fetch_exercises(results_container):
    # Fetch the User and UserAssessment objects
    user = User.objects.get(username='nobel')
    user_assessment = UserAssessment.objects.get(user=user)

    # Split equipment and target area strings into lists
    equipment_list = user_assessment.equipment.split(",") if user_assessment.equipment else []
    target_list = user_assessment.target_area.split(",") if user_assessment.target_area else []

    # Get exercises based on equipment and target area
    equipment_exercises = Exercise.objects.filter(equipment__in=equipment_list).values_list('name', flat=True)
    target_exercises = Exercise.objects.filter(target__icontains__in=target_list).values_list('name', flat=True)

    # Combine and filter unique exercises
    unique_exercises = list(set(equipment_exercises) | set(target_exercises))

    # Filter exercises that match equipment list
    filtered_exercises = [name for name in unique_exercises if Exercise.objects.get(name=name).equipment in equipment_list]

    # Store the filtered exercises in the results container
    results_container.append(filtered_exercises)

# Create a container to hold results
results = []
# Function to generate prompt and call OpenAI API
# Function to generate prompt and call OpenAI API
def generate_workout_plan(results):
    try:
        # Use the fetched exercises in the custom prompt
        user = User.objects.get(username='nobel')
        user_assessment = UserAssessment.objects.get(user=user)

        # Define your custom prompt
        custom_prompt = f"""
        You are a fitness expert. Please generate a detailed 1-day workout plan for a user whose goal is to . The user has the following details:
        - Height: {user_assessment.height} 
        - Weight: {user_assessment.weight} 
        - Gender: {user_assessment.gender}
        - Fitness Goal: {user_assessment.coach.text}
        - Target muscle: {user_assessment.target_area}
        - Equipment Have: {user_assessment.equipment}
        - Training Days per Week: {user_assessment.schedule}

        For each day, include:
        1. Warm-up exercises (5-10 minutes).
        2. Main exercises focusing on the target muscle groups.
        3. Cool down exercises (5-10 minutes).

        Make sure to vary the exercises each day and provide a balanced routine, 
        and also use this list of names only: {results[0] if results else "No results"} and calory burn
        and also provide me only json format.
        """

        # Call the OpenAI API to get the response
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a fitness expert."},
                {"role": "user", "content": custom_prompt}
            ],
            temperature=0.7,
            max_tokens=1500  # Adjust this value based on your needs
        )

        # Extract the response from the API call
        ai_response = response.choices[0].message.content.strip()

        # Debug: Print the AI response
        print("AI Response:")
        print(ai_response)

        # Use regular expression to find the JSON part
        json_match = re.search(r"\{.*\}", ai_response, re.DOTALL)

        if not json_match:
            raise ValueError("No valid JSON found in the AI response.")
        
        json_data = json_match.group()

        # Attempt to parse the AI response as JSON
        workout_plan = json.loads(json_data)

        # Write the JSON data to a file
        with open('workout_plan.json', 'w') as json_file:
            json.dump(workout_plan, json_file, indent=4)

        print("Workout plan saved to workout_plan.json")

    except Exception as e:
        print(f"An error occurred: {e}")

# Create a list to store the results
results = []

# Start a new thread to run the fetch_exercises function
thread1 = threading.Thread(target=fetch_exercises, args=(results,))
thread1.start()
thread1.join()  # Wait for the thread to complete

# Start a new thread to generate the workout plan
thread2 = threading.Thread(target=generate_workout_plan, args=(results,))
thread2.start()
thread2.join()  # Wait for the thread to complete

Exception in thread Thread-6 (fetch_exercises):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/nobel/Desktop/web/Snapfitchat/env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_59699/2001488454.py", line 30, in fetch_exercises
  File "/home/nobel/Desktop/web/Snapfitchat/env/lib/python3.12/site-packages/django/db/models/manager.py", line 87, in manager_method
    return getattr(self.get_queryset(), name)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nobel/Desktop/web/Snapfitchat/env/lib/python3.12/site-packages/django/db/models/query.py", line 1476, in filter
    return self._filter_or_exclude(False, args, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

AI Response:
{
    "Workout Plan": 
    {
        "Day": "Monday",
        "Warm Up": 
        [
            {
                "Exercise": "Jumping Jacks",
                "Duration": "5 minutes",
                "Equipment": "Bodyweight"
            },
            {
                "Exercise": "Jog in place",
                "Duration": "5 minutes",
                "Equipment": "Bodyweight"
            }
        ],
        "Main Workout": 
        [
            {
                "Exercise": "Barbell Bench Press",
                "Sets": 4,
                "Reps": 8,
                "Equipment": "Barbell",
                "Target Muscle": "Chest"
            },
            {
                "Exercise": "Dumbbell Shoulder Press",
                "Sets": 4,
                "Reps": 8,
                "Equipment": "Dumbbells",
                "Target Muscle": "Shoulders"
            },
            {
                "Exercise": "Kettlebell Swing",
                "Sets": 4,
                

In [11]:

!pip install nest_asyncio


import nest_asyncio
nest_asyncio.apply()

from gym.models import NutritionItem, Meal, NutritionPlan
from datetime import date
import asyncio

# Provided JSON data
nutrition_data = {
    "breakfast": [
        {
            "name": "Oatmeal with Berries",
            "calories": 350,
            "protein": "10g",
            "carbs": "50g",
            "fat": "10g"
        },
        {
            "name": "Scrambled Eggs with Spinach",
            "calories": 200,
            "protein": "15g",
            "carbs": "2g",
            "fat": "15g"
        }
    ],
    "lunch": [
        {
            "name": "Grilled Chicken Salad",
            "calories": 450,
            "protein": "40g",
            "carbs": "15g",
            "fat": "20g"
        },
        {
            "name": "Quinoa and Black Beans",
            "calories": 300,
            "protein": "10g",
            "carbs": "50g",
            "fat": "5g"
        }
    ],
    "snack": [
        {
            "name": "Greek Yogurt with Honey",
            "calories": 150,
            "protein": "10g",
            "carbs": "15g",
            "fat": "5g"
        }
    ],
    "dinner": [
        {
            "name": "Baked Salmon with Asparagus",
            "calories": 500,
            "protein": "35g",
            "carbs": "10g",
            "fat": "30g"
        },
        {
            "name": "Brown Rice",
            "calories": 200,
            "protein": "5g",
            "carbs": "45g",
            "fat": "2g"
        }
    ],
    "totalCalories": 2150,
    "macros": {
        "protein": "110g",
        "carbs": "187g",
        "fat": "87g"
    }
}

# Function to create or get NutritionItem
async def get_or_create_nutrition_item(item_data):
    item, created = await NutritionItem.objects.get_or_create(
        name=item_data['name'],
        defaults={
            'calories': item_data['calories'],
            'protein': item_data['protein'],
            'carbs': item_data['carbs'],
            'fat': item_data['fat'],
        }
    )
    return item

# Function to create a Meal
async def create_meal(meal_name, items_data):
    meals = await Meal.objects.create(name=meal_name)
    for item_data in items_data:
        item = await get_or_create_nutrition_item(item_data)
        await meals.items.add(item)  # Ensure you are using async here if using ManyToMany
    return meals

# Main function to create the nutrition plan
async def create_nutrition_plan():
    breakfast = await create_meal('Breakfast', nutrition_data['breakfast'])
    lunch = await create_meal('Lunch', nutrition_data['lunch'])
    snack = await create_meal('Snack', nutrition_data['snack'])
    dinner = await create_meal('Dinner', nutrition_data['dinner'])

    # Create the NutritionPlan
    nutrition_plan = await NutritionPlan.objects.create(
        date=date.today(),
        breakfast=breakfast,
        lunch=lunch,
        snack=snack,
        dinner=dinner,
        total_calories=nutrition_data['totalCalories'],
        total_protein=nutrition_data['macros']['protein'],
        total_carbs=nutrition_data['macros']['carbs'],
        total_fat=nutrition_data['macros']['fat']
    )

    print("Nutrition data saved successfully!")


# Now you can safely call asyncio.run()
asyncio.run(create_nutrition_plan())


RuntimeError: asyncio.run() cannot be called from a running event loop